# Keras全般

* [Home - Keras Documentation](https://keras.io/ja/)
* [kerasのmnistのサンプルを読んでみる - Qiita](https://qiita.com/ash8h/items/29e24fc617b832fba136)

# CNN関連

* [Convolutionalレイヤー - Keras Documentation](https://keras.io/ja/layers/convolutional/)
* [Keras(+Tensorflow)でMNISTしてみる](https://qiita.com/fukuit/items/b3fa460577a0ea139c88)
* [KerasでDeep Learning：CNNを組んでみる](http://tekenuko.hatenablog.com/entry/2017/07/23/195321)
* [KerasでMNIST CNNその1](http://kikei.github.io/ai/2017/08/16/mnist-cnn.html)
* [KerasでMNIST CNNその2](http://kikei.github.io/ai/2018/03/24/mnist-cnn2.html)

# Colaboratoryで実行する場合

Colaboratoryで実行する場合、下記コマンドを実行し、livelossplotをインストールする必要があります。

```
!pip3 install livelossplot
```


In [0]:
!pip3 install livelossplot

# ライブラリのインポート

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from keras.layers.core import Dense, Activation, Dropout, Flatten

# 学習曲線描画
# pip install livelossplot
from livelossplot import PlotLossesKeras

# 共通変数定義

In [0]:
batch_size = 128
num_classes = 10
epochs = 20

# MNISTデータセット取得

In [0]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# テストデータの整形

keras.datasets.mnist.load_data()で取得するMNIST画像データは28px X 28px、0〜255の８ビットグレースケールを元にしたデータになっているため、下記の整形を行います。

* 行列の次元変換： (28, 28) → (784)
* 画素値の正規化： 0〜255 → 0〜1の間の小数

In [0]:
x_train = x_train.reshape(60000, 784) # 2次元配列を1次元に変換
x_test = x_test.reshape(10000, 784)

# 28 x 28の画像がgrayscaleで1chなので、28, 28, 1にreshapeする
print(x_train.shape[0])
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')   # int型をfloat32型に変換
x_test = x_test.astype('float32')

x_train /= 255                        # [0-255]の値を[0.0-1.0]に変換
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# モデル構築

- [ソフトマックス関数](https://mathtrain.jp/softmax)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1,1), padding='valid', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# モデル構造の可視化

[可視化 - Keras Documentation](https://keras.io/ja/visualization/)

## モデルの可視化モジュール

pydotはしばらく前に開発が止まり、pydotplusとpydot-ngに別れたそうです。  
pythonのgraphvizはインターフェースだけのラッパーモジュールなので、実態となるパッケージをインストールする必要があります。

```
$ pip install pydot-ng graphviz
$ sudo apt-get install graphviz
```

In [0]:
# PNG形式でモデルの構造を出力する
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png')

# SVG形式でモデルの構造を出力する
# 環境によっては''''pip install pydot''を実行し、pydotをインストールする必要あり
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# モデルの最適化

In [0]:
### ??? kerasの学習率はどこに効いてくるのかわからなかった。。。
#from keras.callbacks import LearningRateScheduler
#LearningRateScheduler(0.001)
#LearningRateScheduler(0.00000000001)

In [0]:
stack = model.fit(x_train, y_train,  # 画像とラベルデータ
                    batch_size=batch_size,
                    epochs=epochs,     # エポック数の指定
                    verbose=1,         # ログ出力の指定. 0だとログが出ない
                    validation_split=0.1,  # 学習データの1割を検証に使用する
                    callbacks=[PlotLossesKeras()])

# AccuracyとLossの可視化

In [0]:
import matplotlib.pyplot as plt

x = range(epochs)
plt.plot(x, stack.history['acc'], label="acc")
plt.title("accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.plot(x, stack.history['loss'], label="loss")
plt.title("loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

# 評価

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 学習した成果の保存

In [0]:
import os.path

f_model = './model'
if not os.path.exists(f_model):
    os.mkdir(f_model)

json_string = model.to_json()
open(os.path.join(f_model,'tf_keras_mnist_beginner_weights.json'), 'w').write(json_string)

yaml_string = model.to_yaml()
open(os.path.join(f_model,'tf_keras_mnist_beginner_weights.yaml'), 'w').write(yaml_string)

print('save model')
model.save(os.path.join(f_model,'tf_keras_mnist_cnn.h5'))